# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770894450621                   -0.52    9.0
  2   -2.772149032691       -2.90       -1.32    1.0
  3   -2.772170295883       -4.67       -2.44    1.0
  4   -2.772170651671       -6.45       -3.14    1.0
  5   -2.772170722882       -7.15       -4.35    2.0
  6   -2.772170722945      -10.20       -4.49    1.0
  7   -2.772170723013      -10.17       -5.45    1.0
  8   -2.772170723015      -11.75       -5.61    2.0
  9   -2.772170723015      -12.71       -6.29    1.0
 10   -2.772170723015      -13.50       -6.47    1.0
 11   -2.772170723015      -13.91       -6.80    1.0
 12   -2.772170723015      -14.21       -8.65    2.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770645279593                   -0.53    8.0
  2   -2.772050302232       -2.85       -1.30    1.0
  3   -2.772082684792       -4.49       -2.63 

1.7735580041373022

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770392416711                   -0.53    9.0
  2   -2.772050767255       -2.78       -1.29    1.0
  3   -2.772082737385       -4.50       -2.70    1.0
  4   -2.772083415800       -6.17       -3.84    2.0
  5   -2.772083417693       -8.72       -4.40    1.0
  6   -2.772083417801       -9.97       -5.24    1.0
  7   -2.772083417810      -11.03       -5.70    2.0
  8   -2.772083417811      -12.40       -7.37    1.0
  9   -2.772083417811      -14.65       -7.25    3.0
 10   -2.772083417811      -15.35       -7.67    1.0
 11   -2.772083417811      -15.05       -8.36    1.0

Polarizability via ForwardDiff:       1.772534961728542
Polarizability via finite difference: 1.7735580041373022
